In [ ]:
                                                    # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import transformers 
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("../input/seokwon98-embedding-test/kwprojb_data.csv")
#test_data_frame =pd.read_csv("../input/seokwon98-embedding-test/kwprojb_data.csv")
#sample_sub=pd.read_csv("../input/contradictory-my-dear-watson/sample_submission.csv")

In [ ]:
def Utilize_TPUs():  
    """
    Initialize training strategy using TPU if available else using default strategy for CPU and  single GPU
    
    After the TPU is initialized, you can also use manual device placement to place the computation on a single TPU device.

    """
    try:
        
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        REPLICAS = strategy.num_replicas_in_sync
        print("Connected to TPU Successfully:\n TPUs Initialised with Replicas:",REPLICAS)
        
        return strategy
    
    except ValueError:
        
        print("Connection to TPU Falied")
        print("Using default strategy for CPU and single GPU")
        strategy = tf.distribute.get_strategy()
        
        return strategy
    
strategy=Utilize_TPUs()

In [ ]:
df['shop_type'].value_counts()

In [ ]:
A = df[df['shop_type'] == '세탁소'].index

In [ ]:
df.drop(A,axis='index',inplace=True)
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['shop_type'] = le.fit_transform(list(df['shop_type']))

In [ ]:
df

In [ ]:
X_column = ["shop_name"]
y_column = 'shop_type'

In [ ]:
input_data = df[X_column + [y_column]].copy()
input_data

In [ ]:
#the_chosen_one="bert-base-cased"

the_chosen_one="jplu/tf-xlm-roberta-base"

max_len=8
batch_size = 8 * strategy.num_replicas_in_sync
AUTO     = tf.data.experimental.AUTOTUNE
epochs= 20
n_steps = len(df) // batch_size

In [ ]:
#from tensorflow.keras.optimizers import SGD

def model_baseline(strategy,transformer):
    with strategy.scope():
        transformer_encoder = TFAutoModel.from_pretrained(transformer)
        input_layer = Input(shape=(max_len,), dtype=tf.int32, name="input_layer")
        sequence_output = transformer_encoder(input_layer)[0]
        cls_token = sequence_output[:, 0, :]
        output_layer = Dense(60, activation='softmax')(cls_token)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(
            Adam(lr=1e-5), 
            loss='sparse_categorical_crossentropy',
            #optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
            metrics=['accuracy']
        )
        return model
model=model_baseline(strategy,the_chosen_one)

In [ ]:
model.summary()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(the_chosen_one)

In [ ]:
all_data = input_data['shop_name'].values.tolist()

In [ ]:
# 데이터 -> train/valid, test
from sklearn.model_selection import train_test_split

raw_tr_val_X, raw_test_X, tr_val_y, test_y = train_test_split(
    input_data[X_column], input_data[y_column],
    test_size=0.2, shuffle=True, random_state=42, stratify=input_data[y_column])


In [ ]:
# train/valid -> train, valid split
raw_train_X, raw_valid_X, train_y, valid_y = train_test_split(
    raw_tr_val_X, tr_val_y,
    test_size=0.3, shuffle=True, random_state=42, stratify=tr_val_y)


In [ ]:
train_data = raw_train_X['shop_name'].values.tolist()
valid_data = raw_valid_X['shop_name'].values.tolist()
test_data = raw_test_X['shop_name'].values.tolist()


In [ ]:
train_encoded = tokenizer.batch_encode_plus(train_data, pad_to_max_length=True, max_length=max_len)
valid_encoded = tokenizer.batch_encode_plus(valid_data, pad_to_max_length=True, max_length=max_len)
test_encoded = tokenizer.batch_encode_plus(test_data, pad_to_max_length=True, max_length=max_len)


In [ ]:
#tr_val_X, test_X, tr_val_y, test_y = train_test_split(input_data[X_column], input_data[y_column], test_size=0.2, shuffle=True, random_state=42, stratify=input_data[y_column])

In [ ]:
#train_X, valid_X, train_y, valid_y = train_test_split(tr_val_X, tr_val_y, test_size=0.2, shuffle=True, random_state=42, stratify=tr_val_y)

In [ ]:
x_train = train_encoded['input_ids']
x_valid = valid_encoded['input_ids']
x_test = test_encoded['input_ids']


In [ ]:
#train_dataset = (tf.data.Dataset.from_tensor_slices((train_X, train_y)).repeat().shuffle(2048).batch(batch_size).prefetch(AUTO))

#valid_dataset = (tf.data.Dataset.from_tensor_slices((valid_X, valid_y)).batch(batch_size).cache().prefetch(AUTO))

#test_dataset = (tf.data.Dataset.from_tensor_slices(test_X).batch(batch_size))

In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices((x_train, train_y)).repeat().shuffle(2048).batch(batch_size).prefetch(AUTO))

valid_dataset = (tf.data.Dataset.from_tensor_slices((x_valid, valid_y)).batch(batch_size).cache().prefetch(AUTO))

test_dataset = (tf.data.Dataset.from_tensor_slices((x_test, test_y)).batch(batch_size).cache().prefetch(AUTO))


In [ ]:
hist = model.fit(train_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=epochs)

In [ ]:
hist.history

In [ ]:
pred_y_prob = model.predict(test_dataset, verbose=1)


In [ ]:
pred_y = np.argmax(pred_y_prob, axis=1)
pred_y


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(test_y, pred_y))


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,10))
ax = plt.axes()
ax.set_xticks(np.arange(0, epochs, 1))
ax.set_xticklabels(np.arange(1, epochs + 1, 1))
plt.plot(hist.history['loss'], label = 'train loss')
plt.plot(hist.history['val_loss'], label = 'validation loss')
plt.xticks(np.arange(0, epochs, 1))
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc = 'lower right')
plt.show

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,10))
ax = plt.axes()
ax.set_xticks(np.arange(0, epochs, 1))
ax.set_xticklabels(np.arange(1, epochs + 1, 1))
plt.plot(hist.history['accuracy'], label = 'train accuracy')
plt.plot(hist.history['val_accuracy'], label = 'validation accuracy')
plt.xticks(np.arange(0, epochs, 1))
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc = 'lower right')
plt.show

In [ ]:
hist_roberta = hist.history


In [ ]:
import pickle

hist_roberta = hist.history
with open("/trainHistoryDict","wb") as file_pi:
    pickle.dump(hist_roberta, file_pi)

In [ ]:
hist_df = pd.DataFrame(hist.history)

In [ ]:
hist_csv_file = 'roberta-history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
pd.read_csv("./roberta-history.csv")

In [ ]:
with open("/trainHistoryDict","rb") as f:
    hist_roberta = pickle.load(f)

In [ ]:
#-----------ROBERTA MODEL-----------